## *De novo* Canu and Miniasm assembly

###### NOTE: This notebook has high memory and computational requirements

Obtaining the assembled sequence of a complete genome is a complex multi-step task. For *De novo* assembly, the simplest elements of the hierarchy are the reads provided after the sequencing. The next level of hierarchy is the alignment of multiple reads without definite order (i.e. contigs). Finally, the top level of the hierarchy corresponds to the sum of two or more contigs where a (near) complete structure of the genome under study is obtained. Ideally, one expects to obtain a single fragment (contig) for each chromosome or a plasmid that is present in the genome. However, most of the times the assemblies are incomplete, especially when dealing with short reads that can be caused by the process of preparing the material or by the technological limitations. Specifically, when a repeat region is longer than the reads, this will create a single contig in the assembly with multiple connections. ONT provide long reads that can solve this problem albeit the per-base quality is still low (approximately 12-15% error rate).

## *De novo* Canu assembly pipeline

This notebook relies in the popular Canu pipeline (with Racon and Pilon to polish the assembly result) and Miniasm for *de novo* assembly of ONT reads. 

[Canu](https://github.com/marbl/canu) is a popular assembler based on the Celera Assembler that can reliably assemble complete microbial genomes and almost complete eukaryotic chromosomes. Canu has three stages: correction, trimming and assembly. Each stage can be executed independently or in series. Each of the three stages begins by identifying the overlaps between all the pairs of input reads, where they count k-mers in the reads, creating an indexed store of the overlays. From the input reads, the correction stage generates corrected reads. The trimming step trims non-compatible bases and detects fork adapters, chimeric sequences and other anomalies. The assembly stage builds an assembly graph and the contigs. Canu handles the repetitions probabilistically, by statistically filtering the repetitively induced overlays and retrospectively inspecting the graph for possible errors, thereby reducing the possibility of selecting a repetitive k-mer for the overlap. In this way, Canu performs multiple rounds of read and overlapping error correction.

Canu substantially reduces coverage requirements with a low coverage hierarchical assembly. In this case, it is recommended to polish the assembly with short high-quality reads. Canu results are optimal with long-read coverages above 20x. 

Canu works with either FASTA or FASTQ files (compressed and uncompressed), but FASTQ format is needed to run some of the steps and complete the full pipeline. The help page is called with the "canu -h" command. These are the parameters needed for running Canu with our data:

- **-p and -d**: Assembly files prefix and output directory. Both parameters can be the same and output directory doesn't have to exist before execution.

- **-genomeSize**: The estimated genome size. In our case, 2.1 mbp so we write '2.1m'. We can put letter g for gbp or k for kbp as well.

- **-nanopore-raw**: The path to our reads in FASTQ.

Canu auto-detects available resources and will configure job sizes based on the resources and genome size that is being assembled. The following code uses advanced options like 'corMemory' and 'corThreads' to limit the resources in order to make Canu work on a resource-limited environment. These options also prevent Canu to stop the process because of a lack of computation power.

In [1]:
canu -h


usage:   canu [-version] [-citation] \
              [-haplotype | -correct | -trim | -assemble | -trim-assemble] \
              [-s <assembly-specifications-file>] \
               -p <assembly-prefix> \
               -d <assembly-directory> \
               genomeSize=<number>[g|m|k] \
              [other-options] \
              [-haplotype{NAME} illumina.fastq.gz] \
              [-pacbio-raw |
               -pacbio-corrected |
               -nanopore-raw |
               -nanopore-corrected] file1 file2 ...

example: canu -d run1 -p godzilla genomeSize=1g -nanopore-raw reads/*.fasta.gz 


  To restrict canu to only a specific stage, use:
    -haplotype     - generate haplotype-specific reads
    -correct       - generate corrected reads
    -trim          - generate trimmed reads
    -assemble      - generate an assembly
    -trim-assemble - generate trimmed reads and then assemble them

  The assembly is computed in the -d <assembly-directory>, with output files named
  usi

: 1

In [2]:
canu -p sample \
     -d data/sample/canu_output \
     genomeSize=2.1m \
     useGrid=false \
     minReadLength=50 \
     minOverlapLength=50 \
     corMemory=2 \
     corThreads=2 \
     maxMemory=6 \
     stopOnLowCoverage=1 \
     -nanopore-raw data/sample/reads.fastq


-- Canu snapshot v1.8 +44 changes (r9254 a50e26a75ffccc529bd944b7adb291e2b6e1c24b)
--
-- CITATIONS
--
-- Koren S, Walenz BP, Berlin K, Miller JR, Phillippy AM.
-- Canu: scalable and accurate long-read assembly via adaptive k-mer weighting and repeat separation.
-- Genome Res. 2017 May;27(5):722-736.
-- http://doi.org/10.1101/gr.215087.116
-- 
-- Koren S, Rhie A, Walenz BP, Dilthey AT, Bickhart DM, Kingan SB, Hiendleder S, Williams JL, Smith TPL, Phillippy AM.
-- De novo assembly of haplotype-resolved genomes with trio binning.
-- Nat Biotechnol. 2018
-- https//doi.org/10.1038/nbt.4277
-- 
-- Read and contig alignments during correction, consensus and GFA building use:
--   Šošic M, Šikic M.
--   Edlib: a C/C ++ library for fast, exact sequence alignment using edit distance.
--   Bioinformatics. 2017 May 1;33(9):1394-1395.
--   http://doi.org/10.1093/bioinformatics/btw753
-- 
-- Overlaps are generated using:
--   Berlin K, et al.
--   Assembling large genomes with single-molecule sequen

### Racon

[Racon](https://github.com/isovic/racon) is a consensus module to correct raw contigs generated by rapid assembly methods that do not include a consensus step. Canu results in FASTA format and raw reads in FASTQ are used in this step to generate a new FASTA contig file.

Before running Racon, one must align the Canu output to the raw reads file and take the overlaps file in PAF as a parameter for Racon command. This can be done using for example minimap, a superfast aligner for ONT reads.

In [10]:
minimap2 data/sample/canu_output/sample.contigs.fasta \
        data/sample/reads.fastq \
        > data/sample/aligned_reads.paf

[M::mm_idx_gen::0.017*0.70] collected minimizers
[M::mm_idx_gen::0.027*0.89] sorted minimizers
[M::main::0.027*0.88] loaded/built the index for 47 target sequence(s)
[M::mm_mapopt_update::0.030*0.80] mid_occ = 5
[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 47
[M::mm_idx_stat::0.033*0.86] distinct minimizers: 61757 (93.56% are singletons); average occurrences: 1.067; average spacing: 5.340
[M::worker_pipeline::0.533*2.25] mapped 3720 sequences
[M::main] Version: 2.14-r890-dirty
[M::main] CMD: minimap2 data/sample/canu_output/sample.contigs.fasta data/sample/reads.fastq
[M::main] Real time: 0.536 sec; CPU: 1.200 sec; Peak RSS: 0.060 GB


The basic Racon parameters are the following:
- **-t** : Number of threads
- Raw reads (FASTQ)
- Overlaps in (PAF)
- Canu output (FASTA)
- Racon output file name (FASTA)

In [26]:
mkdir -p data/sample/racon_output
racon -t 48 \
     data/sample/reads.fastq \
     data/sample/aligned_reads.paf \
     data/sample/canu_output/sample.contigs.fasta > data/sample/racon_output/sample_racon.contigs.fasta

[racon::Polisher::initialize] loaded target sequences
[racon::Polisher::initialize] loaded sequences
[racon::Polisher::initialize] loaded overlaps
[racon::Polisher::initialize] aligned overlap 785/785
[racon::Polisher::initialize] transformed data into windows
[racon::Polisher::polish] generated consensus for window 726/726


### Pilon (Requires Illumina reads)

[Pilon](https:github.com/broadinstitute/pilon) is a tool that can be used to improve a draft assembly and find variation among species or strains. Pilon maps the reads from Illumina read to an assembled sequence and corrects the errors of the base, and the small insertions and deletions (indels). It requires as input a FASTA file and a BAM file of reads aligned to the input FASTA file. At this point, it will need the Racon contigs file and the reads file (ONT and Illumina reads). The Racon contigs file and the BAM file produced by de alignment of Illumina reads against that contigs file generate the final result of Pilon.

In first place, Illumina reads have to be aligned against the Racon contigs file. BWA is used to index the reference file (Racon contigs) and BWA-MEM is used to perform the alignment of the Illumina reads. In order to have the BAM file required by Pilon, SAMtools is used to convert the BWA output in FASTA to the BAM format.


In [27]:
bwa index data/sample/racon_output/sample_racon.contigs.fasta

[bwa_index] Pack FASTA... 0.00 sec
[bwa_index] Construct BWT for the packed sequence...
[bwa_index] 0.06 seconds elapse.
[bwa_index] Update BWT... 0.00 sec
[bwa_index] Pack forward-only FASTA... 0.00 sec
[bwa_index] Construct SA from BWT and Occ... 0.03 sec
[main] Version: 0.7.15-r1140
[main] CMD: bwa index data/sample/racon_output/sample_racon.contigs.fasta
[main] Real time: 0.130 sec; CPU: 0.096 sec


In [ ]:
mkdir -p data/sample/bwa_output
#Data not included in repository
bwa mem -t 2 \
        data/sample/racon_output/sample_racon.contigs.fasta  \
        data/sample/short/reads_1.fastq.gz \
        data/sample/short/reads_2.fastq.gz \
        | samtools view -S -b -u - | samtools sort - data/sample/bwa_output/bwa_aligned_reads

Before running Pilon, the BWA alignment is indexed using SAMtools:

In [ ]:
samtools index data/sample/bwa_output/bwa_aligned_reads.bam

Pilon is run as a Java .jar executable. Some options can be added to improve the performance of Pilon before specifying the .jar file. Whichever the case, Pilon works better in terms of execution time when it is run on an environment with more available RAM and threads. The parameters used in this Pilon run are the following:

- **--threads**: Number of threads
- **--genome**: The FASTA input file (Racon contigs)
- **--bam**: BAM file (generated by BWA and SAMtools)
- **--outdir** and **--output**: Output directory and filename 

In [ ]:
java -Xmx128g -XX:+UseConcMarkSweepGC \ 
      -XX:-UseGCOverheadLimit \ 
      -jar /home/jovyan/software/pilon/pilon-1.22.jar \ 
      --threads 2 \ 
      --genome  data/sample/racon_output/sample_racon.contigs.fasta \ 
      --bam data/sample/bwa_output/bwa_aligned_reads.bam \ 
      --outdir data/sample/pilon_output \ 
      --output pilon.contigs

## *De novo* Miniasm assembly
 
[Miniasm](https://github.com/lh3/miniasm) is a fast Overlap-Layout-Consensus-based option for *de novo* assembly of noisy long reads. Miniasm builds high-confidence contigs (unitigs) concatenating pieces of read sequences. It takes all read self-mappings as input to output an assembly graph. At least for high-coverage bacterial genomes, Miniasm can generate long contigs from raw ONT reads without error correction. The error rate of the assembly is the same as that of the raw input reads. In this way, Miniasm produces uncontaminated and uncorrected contig sequences from raw read overlays. To reduce the presence of artefacts such as adapters and untrimmed chimeras in the assembly, Miniasm calculates the per base coverage based on good mappings (longer than 2 kb with at least 100 bp non-redundant bases on matching minimizers) versus other reads. 
Furthermore, Miniasm ignores internal matches, eliminates contained reads, and adds overlays to the assembly graph. To avoid multiple edges, Miniasm uses the longest overlay. When the assembly graph is generated, it removes the transitive edges, trims the tilt units composed of few reads, and permits small bubbles to appear. 

Although it cannot produce a high-quality consensus, Miniasm is extremely fast and produces continuous and structurally accurate assemblies, at least for genomes without excessive repetitive sequences. 

Minimap is first used to get an all-vs-all read mappings of the reads:

In [28]:
minimap2  data/sample/reads.fastq data/sample/reads.fastq | gzip -1 > data/sample/reads.paf.gz 

[M::mm_idx_gen::0.936*0.99] collected minimizers
[M::mm_idx_gen::1.112*1.24] sorted minimizers
[M::main::1.112*1.24] loaded/built the index for 3720 target sequence(s)
[M::mm_mapopt_update::1.185*1.22] mid_occ = 24
[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 3720
[M::mm_idx_stat::1.249*1.20] distinct minimizers: 5490363 (89.07% are singletons); average occurrences: 1.187; average spacing: 5.345
[M::worker_pipeline::3.369*2.07] mapped 3720 sequences
[M::main] Version: 2.14-r890-dirty
[M::main] CMD: minimap2 data/sample/reads.fastq data/sample/reads.fastq
[M::main] Real time: 3.391 sec; CPU: 6.988 sec; Peak RSS: 0.288 GB


Miniasm takes the obtained mappings file and the FASTA reads file and generates the final assembly. The assembly algorithm doesn't have a consensus step as it usually needs multiple steps to produce a precise consensus sequence, and that is a computational bottleneck.

In [29]:
miniasm -f data/sample/reads.fastq data/sample/reads.paf.gz  > data/sample/miniasm_assembly.fasta 

[M::main] ===> Step 1: reading read mappings <===
[M::ma_hit_read::0.005*0.75] read 3600 hits; stored 2189 hits and 2189 sequences (33596139 bp)
[M::main] ===> Step 2: 1-pass (crude) read selection <===
[M::ma_hit_sub::0.006*0.72] 0 query sequences remain after sub
[M::ma_hit_cut::0.006*0.72] 0 hits remain after cut
[M::ma_hit_flt::0.006*0.72] 0 hits remain after filtering; crude coverage after filtering: -nan
[M::main] ===> Step 3: 2-pass (fine) read selection <===
[M::ma_hit_sub::0.006*0.72] 0 query sequences remain after sub
[M::ma_hit_cut::0.006*0.72] 0 hits remain after cut
[M::ma_hit_contained::0.006*0.71] 0 sequences and 0 hits remain after containment removal
[M::main] ===> Step 4: graph cleaning <===
[M::ma_sg_gen] read 0 arcs
[M::main] ===> Step 4.1: transitive reduction <===
[M::asg_arc_del_trans] transitively reduced 0 arcs
[M::main] ===> Step 4.2: initial tip cutting and bubble popping <===
[M::asg_cut_tip] cut 0 tips
[M::asg_arc_del_multi] removed 0 multi-arcs
[M::asg_arc

### References

[1] Loman N.J., Quick J. and Simpson J.T. A complete bacterial genome assembled de novo using only nanopore sequencing data. Nature Methods 2015 12:733–735. DOI https://doi.org/10.1101/015552

[2] Li H. Minimap and miniasm: fast mapping and de novo assembly for noisy long sequences. Bioinformatics, Volume 32, Issue 14, 15 July 2016, Pages 2103–2110. DOI https://doi.org/10.1093/bioinformatics/btw152